<a href="https://colab.research.google.com/github/sergeymasl/my_lifehacks/blob/main/Airflow_base_of_knowledge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BaseOperators
разбор кода BaseOperators с примерами как я делал для курса Pandas

[link to BaseOperators](https://airflow.apache.org/docs/apache-airflow/stable/_api/airflow/models/baseoperator/index.html?highlight=baseoperator#airflow.models.baseoperator.BaseOperator)


**отметить `on_failure_callback`**

#### Triger Rules

Triger rules - условие при котором начинаются выполняться tasks

[link on trigger_rules](https://airflow.apache.org/docs/apache-airflow/stable/concepts/dags.html#trigger-rules)

`trigger_rule` может быть указан в параметрах `dag` и тогда он будет применен ко всем операторам (если в операторах не будет прописано другое)

Также параметр `trigger_rule` для каждого оператора

Виды `trigger_rule`:
- `all_success` (default) - все `tasks_parents` были **выполены успешно**
- `all_failed` - все `tasks_parents` были **провалены**
- `all_done` - запускает задачу после `upstream tasks` (`parents') были выполнены **вне зависимости от статуса** успешно, провалено или пропущено 
- `all_skipped` - все задачи были **пропущены**
- `one_failed` - срабатывает **как только одна из задач** была **провалена**
- `one_success` - срабатывает **как только одна из задач** была **успешна**
- `none_failed` - срабатывает когда задачи `parents` были **успешны** или **пропущены**
- `none_failed_min_one_success` - выполняется когда **ни одна из предыдущих** задач **не была провалена**, но **хотя бы одна** была **успешна**
- `none_skipped` - срабатывает если не было **пропусков** предыдущих задач
- `always` - никаких зависимостей задача может выполниться в любой момент

#### Connections

Connections представляют собой подключения к разным источникам данных, задаются они в веб интерфейсе, с указанием `conn_id`. Это `conn_id` в последствии можно вызвывать в тасках

При необходимости к Connections можно получить к данному объекту мы можем обращаться как к обычному словарю.

[link on connections](https://airflow.apache.org/docs/apache-airflow/stable/_api/airflow/hooks/base/index.html)

```python
# Пример доступа к переменной connection
from airflow.hooks.base_hook import BaseHook
host = BaseHook.get_connection("postgres_default").host
pass = BaseHook.get_connection("postgres_default").password
```

#### Variables

Глобальные переменные которые также могут задаваться в веб интерфейсе.

Доступ к переменным осуществляется через `key`.
Глобальные переменные можно:
- получать - `get`
- записывать - `set`
- обновлять - `update`
- удалять - `delete`

[link on variables](https://airflow.apache.org/docs/apache-airflow/stable/_api/airflow/models/variable/index.html)

```python
# Пример доступа к глобальной переменной
from airflow.models import Variable
foo = Variable.get(key = "key")
# присер записи глобальной переменной
a_dict = {'login' : con}
Variable.set(key = 'test_key', value = a_dict, serialize_json = True)
```